# Data Overview and Preparation

In [1]:
import pandas as pd
import numpy as np

## Part 1. DHS wealth index

In [2]:
RAW_DHS_DATA_DIR = 'raw/dhs_wealth_index.csv'
TRAIN_DHS_DATA_DIR = 'extracted/dhs_wealth_index_train.csv'
VAL_DHS_DATA_DIR = 'extracted/dhs_wealth_index_val.csv'
TEST_DHS_DATA_DIR = 'extracted/dhs_wealth_index_test.csv'

df_dhs = pd.read_csv(RAW_DHS_DATA_DIR)
print('total data:',len(df_dhs))
print(df_dhs.columns)

total data: 35235
Index(['cluster', 'svyid', 'wealthpooled', 'wealthpooled5country', 'wealth',
       'iso3', 'hv000', 'year', 'cname', 'country', 'region', 'iso3n',
       'households', 'LATNUM', 'LONGNUM', 'URBAN_RURA'],
      dtype='object')


In [3]:
"""
"wealthpooled": the "asset wealth index" (AWI) of each household, standardized across all surveys, 
  then averaged to the cluster level.
"wealth": "wealth": AWI standardized within each country at the household level, aggregated to the cluster level.  
"""
COLUMN_OF_INTEREST = ['country','year','wealth','wealthpooled', 'households', 'LATNUM','LONGNUM']
df_dhs_relevant = df_dhs.loc[:,COLUMN_OF_INTEREST]
df_dhs_relevant = df_dhs_relevant.dropna()
print(df_dhs_relevant.head())
print('available data:',len(df_dhs_relevant))

  country  year    wealth  wealthpooled  households     LATNUM    LONGNUM
0  Angola  2011  1.713497      2.595618          36 -12.350257  13.534922
1  Angola  2011  1.545335      2.209620          32 -12.360865  13.551494
2  Angola  2011  0.631730      0.906469          36 -12.613421  13.413085
3  Angola  2011  0.826273      1.105359          35 -12.581454  13.397711
4  Angola  2011  1.293282      1.879344          37 -12.578135  13.418748
available data: 27077


In [4]:
print(sorted(set(list(df_dhs_relevant['country']))))

['Angola', 'Benin', 'Burkina Faso', 'Cameroon', "Cote d'Ivoire", 'Democratic Republic of Congo', 'Ethiopia', 'Ghana', 'Guinea', 'Kenya', 'Lesotho', 'Malawi', 'Mali', 'Mozambique', 'Nigeria', 'Rwanda', 'Senegal', 'Sierra Leone', 'Swaziland', 'Tanzania', 'Togo', 'Uganda', 'Zambia', 'Zimbabwe']


In [5]:
# Landsat 7 data is available from 1999-05-28T01:02:17Z - 2022-04-06T00:00:00 
# Landsat 8 data is available from 2013-03-18T15:58:14Z - 2022-06-14T00:00:00
print('list of years:',sorted(set(df_dhs_relevant['year'].to_numpy())))
print('n data 2000 to 2010 (inclusive):',len(df_dhs_relevant[(df_dhs_relevant['year']>=2000) & (df_dhs_relevant['year']<=2010)]))
print('n data 2011 to 2016 (inclusive):',len(df_dhs_relevant[(df_dhs_relevant['year']>=2011) & (df_dhs_relevant['year']<=2016)]))
print()
print('n data 2000 to 2013 (inclusive):',len(df_dhs_relevant[(df_dhs_relevant['year']>=2000) & (df_dhs_relevant['year']<=2013)]))
print('n data 2014 to 2016 (inclusive):',len(df_dhs_relevant[(df_dhs_relevant['year']>=2014) & (df_dhs_relevant['year']<=2016)]))

list of years: [2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016]
n data 2000 to 2010 (inclusive): 10485
n data 2011 to 2016 (inclusive): 16592

n data 2000 to 2013 (inclusive): 18934
n data 2014 to 2016 (inclusive): 8143


In [6]:
def insert_id_column(df):
    idcol = []
    indices = df.index[:]
    for i in indices:
        this_row = df.loc[i]
        country_year_id = '%s_%s_%s'%(str(i),str(this_row['country']),str(this_row['year']))
        idcol.append(country_year_id)
    df.insert(0,'id', idcol)
    return df
df_dhs_relevant = insert_id_column(df_dhs_relevant)
print(df_dhs_relevant.head())

              id country  year    wealth  wealthpooled  households     LATNUM  \
0  0_Angola_2011  Angola  2011  1.713497      2.595618          36 -12.350257   
1  1_Angola_2011  Angola  2011  1.545335      2.209620          32 -12.360865   
2  2_Angola_2011  Angola  2011  0.631730      0.906469          36 -12.613421   
3  3_Angola_2011  Angola  2011  0.826273      1.105359          35 -12.581454   
4  4_Angola_2011  Angola  2011  1.293282      1.879344          37 -12.578135   

     LONGNUM  
0  13.534922  
1  13.551494  
2  13.413085  
3  13.397711  
4  13.418748  


In [7]:
df_dhs_train_val = df_dhs_relevant[(df_dhs_relevant['year']>=2000) & (df_dhs_relevant['year']<=2013)]
df_dhs_test = df_dhs_relevant[(df_dhs_relevant['year']>=2014) & (df_dhs_relevant['year']<=2016)]
print('leng df_dhs_train_val:',len(df_dhs_train_val))
print('leng df_dhs_test:',len(df_dhs_test))

leng df_dhs_train_val: 18934
leng df_dhs_test: 8143


In [8]:
def split_train_val(df,train_probability=0.6):
    assignment_col = np.random.choice(['train','val'],len(df),p=[train_probability,1-train_probability])
    df.insert(0,'assignment',assignment_col)
    df_train = df[df['assignment']=='train'].loc[:,['id']+ COLUMN_OF_INTEREST]
    df_val = df[df['assignment']=='val'].loc[:, ['id']+COLUMN_OF_INTEREST]
    
    return df_train, df_val
df_dhs_train, df_dhs_val = split_train_val(df_dhs_train_val,train_probability=0.6)
print('train:\n',df_dhs_train.head(),'\n leng:', len(df_dhs_train))
print('val:\n',df_dhs_val.head(),'\n leng:',len(df_dhs_val))

train:
               id country  year    wealth  wealthpooled  households     LATNUM  \
0  0_Angola_2011  Angola  2011  1.713497      2.595618          36 -12.350257   
1  1_Angola_2011  Angola  2011  1.545335      2.209620          32 -12.360865   
3  3_Angola_2011  Angola  2011  0.826273      1.105359          35 -12.581454   
4  4_Angola_2011  Angola  2011  1.293282      1.879344          37 -12.578135   
5  5_Angola_2011  Angola  2011  1.153628      1.749317          27 -12.575305   

     LONGNUM  
0  13.534922  
1  13.551494  
3  13.397711  
4  13.418748  
5  13.408575   
 leng: 11358
val:
                 id country  year    wealth  wealthpooled  households  \
2    2_Angola_2011  Angola  2011  0.631730      0.906469          36   
6    6_Angola_2011  Angola  2011  0.072375      0.182361          33   
11  11_Angola_2011  Angola  2011  0.033149     -0.052652          34   
12  12_Angola_2011  Angola  2011 -0.123748     -0.073470          36   
13  13_Angola_2011  Angola  2011  0

In [9]:
df_dhs_train.to_csv(TRAIN_DHS_DATA_DIR)
df_dhs_val.to_csv(VAL_DHS_DATA_DIR)
df_dhs_test.to_csv(TEST_DHS_DATA_DIR)

## Part 2. LSMS Delta Pair

In [10]:
RAW_LSMS_DATA_DIR = 'raw/lsms_wealth_index.csv'
TRAIN_LSMS_DATA_DIR = 'extracted/lsms_wealth_index_train.csv'
VAL_LSMS_DATA_DIR = 'extracted/lsms_wealth_index_val.csv'
TEST_LSMS_DATA_DIR = 'extracted/lsms_wealth_index_test.csv'

"""
index_all comes from lsms_labels_agg.csv
  "index" column computed as PCA over all LSMS data over the 5 countries (Ethiopia, Malawi, Nigeria, Tanzania, Uganda);
  index computed over households in all 3020 LSMS villages
index_rep comes from lsms_labels_index_agg_geolocated.csv
  updated index, with more migrant households removed
"""

df_lsms = pd.read_csv(RAW_LSMS_DATA_DIR)
print('total data:',len(df_lsms))
print(df_lsms.columns)

total data: 3020
Index(['lat', 'lon', 'year', 'country', 'index_all', 'ea_id', 'index_rep',
       'et_index', 'n', 'geolev1', 'geolev2', 'rooms', 'electric', 'phone',
       'radio', 'tv', 'auto', 'floor_qual', 'toilet_qual', 'watsup_qual'],
      dtype='object')


In [11]:
LSMS_COLUMNS = ['lat', 'lon', 'year', 'country', 'index_all', 'index_rep']
df_lsms_rel = df_lsms.loc[:,LSMS_COLUMNS]
df_lsms_rel = df_lsms_rel.dropna()
print(df_lsms_rel.head())
print('available data:',len(df_lsms_rel))

        lat        lon  year country  index_all  index_rep
0 -0.292248  31.478722  2005  uganda   0.045405   0.042136
1 -0.292248  31.478722  2009  uganda  -0.333650  -0.326794
2 -0.292248  31.478722  2013  uganda  -0.386531  -0.373920
3 -0.439120  31.711950  2005  uganda   0.310049   0.301829
4 -0.439120  31.711950  2009  uganda   0.085589   0.090705
available data: 2915


In [12]:
print(sorted(set(list(df_lsms_rel['country']))))
print('list of year:',sorted(set(df_lsms_rel['year'].to_numpy())))
print()
print('n data 2000 to 2013 (inclusive):',len(df_lsms_rel[(df_lsms_rel['year']>=2000) & (df_lsms_rel['year']<=2013)]))
print('n data 2014 to 2016 (inclusive):',len(df_lsms_rel[(df_lsms_rel['year']>=2014) & (df_lsms_rel['year']<=2016)]))

['ethiopia', 'malawi', 'nigeria', 'tanzania', 'uganda']
list of year: [2005, 2008, 2009, 2010, 2011, 2012, 2013, 2015, 2016]

n data 2000 to 2013 (inclusive): 2006
n data 2014 to 2016 (inclusive): 909


In [13]:
df_lsms_rel = insert_id_column(df_lsms_rel)
print(df_lsms_rel.head())

              id       lat        lon  year country  index_all  index_rep
0  0_uganda_2005 -0.292248  31.478722  2005  uganda   0.045405   0.042136
1  1_uganda_2009 -0.292248  31.478722  2009  uganda  -0.333650  -0.326794
2  2_uganda_2013 -0.292248  31.478722  2013  uganda  -0.386531  -0.373920
3  3_uganda_2005 -0.439120  31.711950  2005  uganda   0.310049   0.301829
4  4_uganda_2009 -0.439120  31.711950  2009  uganda   0.085589   0.090705


In [14]:
def split_train_val_test(df, train_probability=0.6):
    df_train_val = df[(df['year']>=2000) & (df['year']<=2013)]
    assignment_col = np.random.choice(['train','val'],len(df_train_val),p=[train_probability,1-train_probability])
    df_train_val.insert(0,'assignment',assignment_col)
    
    df_train = df_train_val[df_train_val['assignment']=='train']
    df_train = df_train.loc[:,['id']+LSMS_COLUMNS]
    df_val = df_train_val[df_train_val['assignment']=='val']
    df_val = df_val.loc[:,['id']+LSMS_COLUMNS]
    
    df_test = df[(df['year']>=2014) & (df['year']<=2016)]
    return df_train, df_val, df_test

df_train, df_val, df_test =split_train_val_test(df_lsms_rel)

print('n train/val, test',len(df_train), len(df_val), len(df_test))
print(df_train.head())
print(df_val.head())
print(df_test.head())

n train/val, test 1185 821 909
                id       lat        lon  year country  index_all  index_rep
0    0_uganda_2005 -0.292248  31.478722  2005  uganda   0.045405   0.042136
5    5_uganda_2013 -0.439120  31.711950  2013  uganda   0.317944   0.340910
7    7_uganda_2009 -0.503361  31.616070  2009  uganda   0.065410   0.081197
10  10_uganda_2009 -0.508831  31.333286  2009  uganda  -0.531709  -0.525071
12  12_uganda_2005 -0.518612  31.194437  2005  uganda   0.135682   0.130125
              id       lat        lon  year country  index_all  index_rep
1  1_uganda_2009 -0.292248  31.478722  2009  uganda  -0.333650  -0.326794
2  2_uganda_2013 -0.292248  31.478722  2013  uganda  -0.386531  -0.373920
3  3_uganda_2005 -0.439120  31.711950  2005  uganda   0.310049   0.301829
4  4_uganda_2009 -0.439120  31.711950  2009  uganda   0.085589   0.090705
6  6_uganda_2005 -0.503361  31.616070  2005  uganda   0.579641   0.576637
                  id      lat        lon  year country  index_all  in

In [15]:
df_train.to_csv(TRAIN_LSMS_DATA_DIR)
df_val.to_csv(VAL_LSMS_DATA_DIR) 
df_test.to_csv(TEST_LSMS_DATA_DIR)